In [23]:
import pandas as pd
from bs4 import BeautifulSoup
import re

with open(r"C:\Users\s3309\Desktop\Investment\Material\tifrs-fr1-m1-ci-cr-2330-2022Q2.html", encoding="UTF-8") as f:
    data=f.read()
soup = BeautifulSoup(data,'html.parser')




data_all=[[] for i in range(5)]

for sht in range(3):
    sheet=soup.find_all('table')[sht]
    cate=sheet.find_all('tr')[0].find_all('th')[0].find(class_='en').text.strip()

    i=1
    while i< len(sheet.find_all('tr')):
        print(i)

        if i==1:
            time_window=[]
            for time in sheet.find_all('tr')[i].find_all('th')[2:]:
                time_window.append(time.find(class_='en').text)

        elif sheet.find_all('tr')[i].find_all('td')[0].text=='' and re.search(r'\d',sheet.find_all('tr')[i+1].find_all('td')[0].text):
            sub_cate=sheet.find_all('tr')[i].find_all('td')[1].find(class_='en').text.strip()

        elif re.search(r'\d',sheet.find_all('tr')[i].find_all('td')[0].text):

            for j,n in enumerate(sheet.find_all('tr')[i].find_all('td')[2:]):

                data_all[0].append(cate)
                data_all[1].append(sub_cate)
                data_all[2].append(adsheet.find_all('tr')[i].find_all('td')[1].find(class_='en').text.strip())
                data_all[3].append(time_window[j])
                data_all[4].append(n.text)



        i+=1





1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79


In [24]:
table=pd.DataFrame()
for i,col in enumerate(['Category','Sub-Category','Subject','Period','$']):

    table[col]=data_all[i]

In [13]:
'   123213  '.strip()

'123213'

In [5]:
%pip install ipykernel

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: colorama in c:\users\s3309\appdata\local\programs\python\python39\dlenv\lib\site-packages (from ipython>=5.0.0->ipykernel) (0.4.4)


You should consider upgrading via the 'C:\Users\s3309\AppData\Local\Programs\Python\Python39\dlenv\Scripts\python.exe -m pip install --upgrade pip' command.
